## Maize Sam Demo
### This Notebook shows the Demo how our pipeline find number of the kernel per row from croped corn ear
### run on all hybrid images

In [1]:
import sys
sys.path.append("./utils")
sys.path.append("./Models")

from corn_path_finder import CornPathFinder
from segment_any_corn import SegmentAnyCorn
from post_processing_corn import PostprocessingCorn
from corn_data import CornData
import matplotlib.pyplot as plt
import sys

from visualization import show_path_on_corn

In [2]:
HYBRID_FOLDER_PATH = "./Data/hybrid_maize/Croppe"

In [3]:
import os

def get_image_paths(root_folder):
    # List of image extensions
    image_extensions = ['.png']
    
    # List to store image paths
    maize_ear_image_paths = []

    # Walk through the directory
    for subdir, _, files in os.walk(root_folder):
        for file in files:
            # Check if the file is an image
            if any(file.lower().endswith(ext) for ext in image_extensions):
                # Add the file path to the list
                maize_ear_image_paths.append(os.path.join(subdir, file))
    
    return maize_ear_image_paths
image_paths = get_image_paths(HYBRID_FOLDER_PATH)


In [4]:
def process_image(image_path):
    # Load the image
    data = CornData(image_path)
    
    # Segment each kernel of corn by using SAM model
    seg = SegmentAnyCorn(data, checkpoint="./Models/saved_weight/sam_vit_h_4b8939.pth")
    seg.instance_segmention_of_cob()
    
    # Apply post processing on the segmented image
    post = PostprocessingCorn(data)
    post.check_iou_area()
    post.calculate_center_bondingbox()
    
    # Find the path of the corn ear after post processing
    path = CornPathFinder(data)
    path.make_path()
    
    
    dir_path = os.path.dirname(image_path)
    file_name = os.path.basename(image_path).split(".")[0]

    pkl_path = os.path.join(dir_path, file_name + ".pkl")


    # Save the data to a file
    data.save_to_file(pkl_path)

In [5]:
process_image(image_paths[0])

Segmentation is ended for the image in ./Data/hybrid_maize/Croped/22-A-1585227/22-A-1585227-24 OP.png
the central path length is 44
the right path length is 41
An unexpected error occurred: No path to 8.
An unexpected error occurred: object of type 'NoneType' has no len()


In [ ]:
image_paths[0]

In [ ]:
import concurrent.futures

# Run the tasks in parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_image, image_paths))

In [ ]:
pkl_file = "./Data/hybrid_maize/Croped/22-A-1585227/22-A-1585227-24 OP.pkl"